In [ ]:
# !pip install linearmodels

In [ ]:
import os
import re

import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go

import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels.formula.api as smf
from linearmodels.panel import PanelOLS

import warnings
# Suppress specific UserWarning from scikit-learn
warnings.filterwarnings(action='ignore', category=UserWarning, module='sklearn')


##################
# Configurations #
##################
# Pandas
pd.set_option("display.max_rows", 50, "display.max_columns", None, "display.width", 200)
pd.options.display.precision = 3  # 3 significant digits

# Plotly
plotly.io.renderers.default = "colab"
plotly.io.templates.default = "plotly_white"
pd.options.plotting.backend = "plotly"

EPSILON = 1E-6

import warnings
warnings.filterwarnings("ignore", message="invalid value encountered in double_scalars")

In [ ]:
def add_y_equal_x_line(fig, x_min=None, x_max=None):
    """Make a plotly scatterplot figure scale ratio 1:1 and add dashed y=x line."""
    fig.update_yaxes(
        scaleanchor = "x",
        scaleratio = 1,
        )

    if x_min is None:
        x_min, x_max = fig.layout.xaxis.range[0], fig.layout.xaxis.range[1]
    min_max = (x_min, x_max)

    fig.update_layout(width=500)
    fig.add_trace(go.Scatter(x=min_max, y=min_max,
                             mode='lines',
                             line=dict(color='gray', dash='dash', width=2),
                             opacity=0.5,
                             name='y=x'))
    return fig

def is_fe_var(variable_name):
    """Whether the variable name indicates a Fixed Effect, C(<something>)."""
    return bool(re.match(r'C\(.*\)', variable_name))


def has_fe(model):
    """Whether the model has Fixed Effects."""
    exog_names = model.exog_names
    if any([is_fe_var(exog_name) for exog_name in exog_names]):
        return 'Yes'
    else:
        return 'No'


def reg_summary(fitted_model, exclude_fe=True):
    """Regression summary as a DataFrame, not the statsmodels returned object.

    :param fitted_model: statsmodels regression model, with .fit()
    :param exclude_fe: Whether to exclude Fixed Effects variables in the returned coefficients table.

    :return: DataFrame with coefficients, standard errors, p-values, CIs etc.
    """
    try:
        sm_summary = fitted_model.summary()
    except TypeError:
        sm_summary = fitted_model.summary
    summary_df = pd.DataFrame(sm_summary.tables[1][1:], columns=sm_summary.tables[1][0])
    summary_df.columns = summary_df.columns.astype(str)
    summary_df.rename(columns={'': 'variable'}, inplace=True)
    # Change data types.
    summary_df['variable'] = summary_df['variable'].astype(str)
    for col_name in summary_df.columns[1:]:
        summary_df[col_name] = summary_df[col_name].astype(str).astype(float)
    if exclude_fe:
        summary_df = summary_df[~summary_df['variable'].apply(is_fe_var)]

    return summary_df

# Simulation and prediction funcs

In [ ]:
def simulate_outcomes(n_experiment_persons=13000, mean_outcome=3200, std_dev_outcome=1400):
    """Simulate log-normal outcomes for n_experiment_persons with mean and std_dev_outcome."""
    # actual_outcomes = np.random.normal(mean_outcome, std_dev_outcome, n_experiment_persons)

    # mean = exp(mu + sigma^2/2)
    # var = [exp(sigma^2) - 1] * exp(2*mu + sigma^2)
    # -> mu = log(mean^2 / sqrt(var + mean^2))
    # -> sigma^2 = log(var / mean^2 + 1)
    mu = np.log(mean_outcome**2 / np.sqrt(std_dev_outcome**2 + mean_outcome**2))
    sigma = np.sqrt(np.log(std_dev_outcome**2 / mean_outcome**2 + 1))
    actual_outcomes = np.random.lognormal(mu, sigma, n_experiment_persons)

    return actual_outcomes

# actual_outcomes = simulate_outcomes()
# px.histogram(actual_outcomes, nbins=100, title="Histogram of Actual outcomes").show()

In [ ]:
def simulate_experiment_data(n_experiment_persons=13000, n_collected_outcomes=1200, mean_outcome=3200, std_dev_outcome=1400,
                             treatment_effect=200, noise_sd=3000):
    data_by_person = pd.DataFrame({'actual_outcome': simulate_outcomes(n_experiment_persons, mean_outcome, std_dev_outcome)})
    data_by_person['is_treatment'] = np.random.rand(n_experiment_persons) < 0.5
    data_by_person['actual_outcome'] += data_by_person['is_treatment'] * treatment_effect
    # Gives about SD=600 for the predictions, where for the actual outcomes, the SD is 1400.
    data_by_person['outcome_noisy_predictor'] = data_by_person['actual_outcome'] + np.random.normal(0, noise_sd, n_experiment_persons)

    data_by_person['is_test'] = np.random.rand(n_experiment_persons) < (n_experiment_persons - n_collected_outcomes)/n_experiment_persons
    return data_by_person

def predict_outcomes(data_by_person, verbose=False):
    """Predict actual_outcome from outcome_noisy_predictor using sklearn LinearRegression, and two other methods."""
    # Reshape data for scikit-learn
    # Features matrix X needs to be 2D (n_samples, n_features)
    X = data_by_person[['outcome_noisy_predictor']].values
    y = data_by_person['actual_outcome']  # Target variable

    X_train = data_by_person.query('not is_test')[['outcome_noisy_predictor']].values
    X_test = data_by_person.query('is_test')[['outcome_noisy_predictor']].values
    y_train = data_by_person.query('not is_test')['actual_outcome'].values
    y_test = data_by_person.query('is_test')['actual_outcome'].values

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=(n_experiment_persons - n_collected_outcomes)/n_experiment_persons, random_state=42)

    # Initialize the model
    model = LinearRegression()

    # Fit the model to the training data
    model.fit(X_train, y_train)

    # Predict on the testing set
    y_pred = model.predict(X_test)
    data_by_person['predicted_outcome'] = model.predict(data_by_person[['outcome_noisy_predictor']])

    data_by_person['mean_outcome'] = mean_outcome = data_by_person['actual_outcome'].mean()
    data_by_person['compressed_outcome'] = mean_outcome + (data_by_person['actual_outcome'] - mean_outcome) * 0.3

    # Evaluate the model
    rmse_pred = np.sqrt(mean_squared_error(data_by_person['actual_outcome'], data_by_person['predicted_outcome']))

    # Evaluate benchmark constant model
    rmse_const = np.sqrt(mean_squared_error(data_by_person['actual_outcome'], data_by_person['mean_outcome']))

    # Evaluate compressed y
    rmse_compressed = np.sqrt(mean_squared_error(data_by_person['actual_outcome'], data_by_person['compressed_outcome']))
    if verbose:
        print(f"rmse_pred: {rmse_pred}")
        print(f"rmse_const: {rmse_const}")
        print(f"rmse_compressed: {rmse_compressed}")

    # px.histogram(pd.concat([pd.DataFrame({'outcome': y_pred, 'source': 'predicted'}), pd.DataFrame({'outcome': y_test, 'source': 'actual'})]),
    #             x='outcome', color='source', histnorm='probability', barmode='overlay', opacity=0.5, title="Histogram of Predicted and Actual outcomes").show()

    return data_by_person

## Experiments

In [ ]:
data_by_person = simulate_experiment_data(treatment_effect=200, noise_sd=3000)
data_by_person = predict_outcomes(data_by_person)

# px.histogram(pd.concat([pd.DataFrame({'outcome': data_by_person['actual_outcome'], 'source': 'actual'}), pd.DataFrame({'outcome': data_by_person['predicted_outcome'], 'source': 'predicted'})]),
#              x='outcome', color='source', histnorm='probability', barmode='overlay', opacity=0.5, title="Histogram of Predicted and Actual outcomes", width=1000).show()

# print(data_by_person[['actual_outcome', 'predicted_outcome']].describe())

print(smf.ols('predicted_outcome ~ is_treatment', data=data_by_person).fit().summary())
print('\n'+'-'*120+'\n')

data_by_person['predicted_outcome'] = data_by_person['outcome_noisy_predictor']

print(smf.ols('predicted_outcome ~ is_treatment', data=data_by_person).fit().summary())
print('\n'+'-'*120+'\n')

In [ ]:

# fig = data_by_person.plot.scatter(x='outcome_noisy_predictor', y='actual_outcome', title="Actual outcomes vs. Noisy Predictor", opacity=0.03)
# add_y_equal_x_line(fig, *data_by_person['actual_outcome'].agg(['min', 'max']).tolist()).show()

# print(smf.ols('actual_outcome ~ is_treatment', data=data_by_person).fit().summary())
# print('\n'+'-'*120+'\n')
# print(smf.ols('predicted_outcome ~ is_treatment', data=data_by_person).fit().summary())
# print('\n'+'-'*120+'\n')
# print(smf.ols('compressed_outcome ~ is_treatment', data=data_by_person).fit().summary())

TREATMENT_EFFECT = 200

res_list = []
for i in range(100):
    noise_sd = np.clip(np.random.normal(3000, 1000), 500, np.inf)
    data_by_person = simulate_experiment_data(treatment_effect=TREATMENT_EFFECT, noise_sd=noise_sd)
    data_by_person = predict_outcomes(data_by_person)
    # data_by_person['predicted_outcome'] = data_by_person['outcome_noisy_predictor']

    r2 = r2_score(data_by_person['actual_outcome'], data_by_person['predicted_outcome'])
    rmse = mean_squared_error(data_by_person['actual_outcome'], data_by_person['predicted_outcome'])**0.5
    pred_sd = np.std(data_by_person['predicted_outcome'])
    actual_sd = np.std(data_by_person['actual_outcome'])

    fitted_model = smf.ols('predicted_outcome ~ is_treatment', data=data_by_person).fit()
    coef, std_err, p_value = reg_summary(fitted_model)[['coef', 'std err', 'P>|t|']].iloc[1].tolist()
    coef, std_err, p_value

    res_list.append({'coef': coef, 'std_err': std_err, 'p_value': p_value, 'treatment_effect': 200, 'noise_sd': noise_sd, 'pred_sd': pred_sd, 'actual_sd': actual_sd, 'r2': r2, 'rmse': rmse})

res_df = pd.DataFrame(res_list)
res_df['pred_compression'] = res_df['pred_sd'] / res_df['actual_sd']
res_df['coef_compression'] = res_df['coef'] / TREATMENT_EFFECT

res_df

In [ ]:
res_df.plot.scatter(x='r2', y='pred_sd')
res_df.plot.scatter(x='r2', y='noise_sd')
res_df.plot.scatter(x='r2', y='pred_compression')
res_df.plot.scatter(x='r2', y=['coef', 'std_err'])
res_df.plot.scatter(x='r2', y='p_value')

fig = res_df.plot.scatter(x='pred_compression', y='coef_compression', title="Compression of Predicted SD vs. Coefficient")
add_y_equal_x_line(fig, 0, 1).show()

fig = res_df.plot.scatter(x='r2', y='coef_compression', title="R2 vs. Coefficient Compression")
add_y_equal_x_line(fig, 0, 1).show()


# Prediction decomposition

## Functions

In [ ]:
def generate_data_by_person_time(n_experiment_persons=13000, mean_mu=3200, std_dev_mu=1400,
    treatment_effect=200, person_time_error_sd=400, n_time_periods=2) -> pd.DataFrame:
    """Simulates data by person and time period, with person fixed-effects, treatment effect, and noise.

    Returns:
        DataFrame with columns:
        'person_id', 'time' (0 to n_time_periods-1), # DataFrame is unique by 'person_id' and 'time'.
        'intercept', 'mu' (person fixed-effects), 'is_treatment' (random bernoulli(0.5) per person),
        'treatment_effect' (0 for untreated, treatment_effect for treated),
        'error_epsilon' (noise by person-time),
        'actual_outcome' (=intercept + mu + treatment_effect + error_epsilon)
    """
    data_by_person = pd.DataFrame({'mu': simulate_outcomes(n_experiment_persons, mean_mu, std_dev_mu), 'person_id': np.arange(n_experiment_persons)})
    # Make mu have mean 0. Export its mean to 'intercept'.
    intercept = data_by_person['mu'].mean()
    data_by_person['intercept'] = intercept
    data_by_person['mu'] -= intercept

    data_by_person['is_treatment'] = np.random.rand(n_experiment_persons) < 0.5
    data_by_person_time = data_by_person.merge(pd.DataFrame({'time': np.arange(n_time_periods)}), how='cross')
    gamma = treatment_effect
    data_by_person_time['treatment_effect'] = data_by_person_time['is_treatment'] * gamma
    data_by_person_time['error_epsilon'] = np.random.normal(0, person_time_error_sd, len(data_by_person_time))
    data_by_person_time['actual_outcome'] = data_by_person_time['intercept'] + data_by_person_time['mu'] + data_by_person_time['treatment_effect'] + data_by_person_time['error_epsilon']

    return data_by_person_time

def add_imperfect_prediction(data_by_person_time, eta_mu, eta_treatment, eta_epsilon, pred_error_nu_sd):
    """Adds column 'predicted_outcome' do the DataFrame, with imperfect prediction with the given eta's.

    predicted_outcome = intercept + eta_mu * mu + eta_treatment * treatment_effect + eta_epsilon * error_epsilon + pred_error_nu
    """
    data_by_person_time['pred_error_nu'] = np.random.normal(0, pred_error_nu_sd, len(data_by_person_time))
    data_by_person_time['predicted_outcome'] = (
        data_by_person_time['intercept'] +
        eta_mu * data_by_person_time['mu'] +
        eta_treatment * data_by_person_time['treatment_effect'] +
        eta_epsilon * data_by_person_time['error_epsilon'] +
        data_by_person_time['pred_error_nu']
        )

    return data_by_person_time

def get_diffs_by_person(data_by_person_time):
    """Gets actual(time1) - actual(time0) and the same for predicted. By person. Assumes only two time periods."""
    data_by_person_time = data_by_person_time.copy()
    assert data_by_person_time['time'].nunique() == 2
    # grouped_by_person = data_by_person_time.sort_values(['person_id', 'time']).groupby('person_id')
    # grouped_by_person['actual_outcome'].nth(1)

    data_by_person_time.set_index(['person_id', 'time'], inplace=True)

    # Unstack the 'time' level to pivot the data
    data_by_person_time_unstacked = data_by_person_time[['actual_outcome', 'predicted_outcome']].unstack(level='time')

    # Calculate the difference between the times for 'outcome' and 'pred'
    diff_outcome_by_person = data_by_person_time_unstacked['actual_outcome'].diff(axis=1).dropna(axis=1)  # Drops the first column of NaNs after diff
    diff_outcome_by_person.columns = ['actual_outcome_diff']

    diff_pred_outcome_by_person = data_by_person_time_unstacked['predicted_outcome'].diff(axis=1).dropna(axis=1)
    diff_pred_outcome_by_person.columns = ['predicted_outcome_diff']

    # Combine the differences into a single DataFrame
    pred_actual_by_person = pd.concat([diff_outcome_by_person, diff_pred_outcome_by_person], axis=1).reset_index()
    return pred_actual_by_person

def test_diff_prediction_diff_actual_regression(data_by_person_time) -> (float, float):
    """Regresses diff of predictions to diff of actuals. Returns coefficient, std error, and R^2."""
    pred_actual_by_person = get_diffs_by_person(data_by_person_time)

    fitted_model = smf.ols('predicted_outcome_diff ~ actual_outcome_diff - 1', data=pred_actual_by_person).fit()
    summary_df = reg_summary(fitted_model)
    coef, std_err = summary_df.query('variable == "actual_outcome_diff"')[['coef', 'std err']].iloc[0].tolist()
    return (coef,
            std_err,
            fitted_model.rsquared)

def test_within_person_prediction(data_by_person_time) -> float:
    """Regress predicted outcome on actual outcome with person fixed-effects.

    predictedOutcome_i,t ~ acutalOutcome_i,t + personFixedEffects_i
    This is the generalization of test_diff_prediction_diff_actual_regression.

    Returns coefficient, standard error, and R^2.
    """
    data_by_person_time = data_by_person_time.copy()
    # Convert data to panel format
    panel_data = data_by_person_time.set_index(['person_id', 'time'])

    # Define the model
    model = PanelOLS.from_formula('predicted_outcome ~ actual_outcome + EntityEffects', panel_data)

    # Fit the model
    try:
        fitted_model = model.fit()
    except ZeroDivisionError:
        if data_by_person_time.groupby('person_id')['predicted_outcome'].var().max() == 0:
            # Predicted Outcome is constant within person, regression gives an error
            # so return NaN for coef and standard error, r_squared=1.
            r_squared = 1
            return (np.nan, np.nan, r_squared)
        elif (data_by_person_time['actual_outcome'] == data_by_person_time['predicted_outcome']).all():
            # actual_outcome == predicted_outcome, this causes an error for PanelOLS.
            return (1, 0, 1)
        else:
            # Something unexpected.
            raise

    summary_df = reg_summary(fitted_model)
    coef, std_err = summary_df.query('variable == "actual_outcome"')[['Parameter', 'Std. Err.']].iloc[0].tolist()
    return (coef,
            std_err,
            fitted_model.rsquared)

def test_person_fixed_effects_explained_variance(data_by_person_time) -> float:
    """How much of the variance in actual outcome is explained by person fixed effects.

    This is even more than (st_dev_mu**2 + st_dev_treatment**2) / (st_dev_mu**2 + st_dev_treatment**2 + st_dev_epsilon**2), because it's the empirical mean of each person.
    """
    data_by_person_time = data_by_person_time.copy()
    # Calculating explicitly, because the regression takes too long.
    mean_outcome_by_person = data_by_person_time.groupby('person_id')['actual_outcome'].mean().rename('person_mean_outcome').reset_index()

    data_by_person_time = data_by_person_time.merge(mean_outcome_by_person, on='person_id')
    return r2_score(data_by_person_time['actual_outcome'], data_by_person_time['person_mean_outcome'])

def test_treatment_effect_on_predictions(data_by_person_time) -> tuple[float]:
    """Treatment effect regression results on predicted outcome."""
    fitted_model = smf.ols('predicted_outcome ~ is_treatment', data=data_by_person_time.astype({'is_treatment': int})).fit()
    summary_df = reg_summary(fitted_model)
    coef, std_err, t_stat, p_value = summary_df.query('variable == "is_treatment"')[['coef', 'std err', 't', 'P>|t|']].iloc[0]
    return coef, std_err, t_stat

def calc_prediction_r2(data_by_person_time) -> float:
    return r2_score(data_by_person_time['actual_outcome'], data_by_person_time['predicted_outcome'])

def calc_prediction_stats(data_by_person_time) -> pd.DataFrame:
    """Returns DataFrame with various summary stats of the prediction performance/stats in various tests."""
    result_dict = {}

    result_dict['prediction_r2'] = calc_prediction_r2(data_by_person_time)
    # result_dict['diff_pred_coef'], result_dict['diff_pred_std_err'], result_dict['diff_pred_r2'] = test_diff_prediction_diff_actual_regression(data_by_person_time)
    result_dict['diff_pred_coef'], result_dict['diff_pred_std_err'], result_dict['diff_pred_r2'] = test_within_person_prediction(data_by_person_time)

    result_dict['actual_outcome_sd'] = data_by_person_time['actual_outcome'].std()
    result_dict['predicted_outcome_sd'] = data_by_person_time['predicted_outcome'].std()
    result_dict['compression'] = result_dict['predicted_outcome_sd'] / result_dict['actual_outcome_sd']

    # This is even more than 1 - (st_dev_epsilon**2) / (st_dev_mu**2 + st_dev_treatment**2 + st_dev_epsilon**2), because it's the empirical mean of each person.
    result_dict['var_explained_person'] = test_person_fixed_effects_explained_variance(data_by_person_time)
    # (1400**2 + 100**2) / (1400**2 + 100**2 + 400**2)

    result_dict['treatment_coef'], result_dict['treatment_se'], result_dict['treatment_t'] = test_treatment_effect_on_predictions(data_by_person_time)

    return pd.DataFrame([result_dict])

def estimate_eta_mu(data_by_person_time):
    """Estimate eta_mu using a formula of covariances which should in theory equal it.

    \frac{\textrm{Cov}[\textrm{predOut}_{i,t}, \textrm{actualOut}_{i,t}] - \frac{1}{2}\textrm{Cov}[\Delta\textrm{predOut}_i, \Delta\textrm{actualOut}_i]}{\textrm{Var}[\textrm{actualOut}_{i,t}] - \frac{1}{2}\textrm{Var}[\Delta\textrm{actualOut}_i]}
    """
    # TODO: there are cases where this fails, probably pathologies with outcomes being constant within person.
    # TODO: wrap this with a function which performs bootstrap to get the standard errors.
    covariances = data_by_person_time[['predicted_outcome', 'actual_outcome']].cov()
    deltas = data_by_person_time.groupby('person_id')[['predicted_outcome', 'actual_outcome']].diff(1).dropna()
    delta_covariances = deltas.cov()
    eta_mu_hat = (covariances.iloc[0, 1] - 0.5 * delta_covariances.iloc[0, 1]) / (covariances.iloc[1, 1] - 0.5 * delta_covariances.iloc[1, 1])
    return eta_mu_hat

## Sanity Checks / tests

In [ ]:
# FE Regression is equivalent to diff-vs-diff regression, for two time periods.
if False:
    n_experiment_persons=13000; mean_mu=3200; std_dev_mu=1400; treatment_effect=200; person_time_error_sd=600
    data_by_person_time = generate_data_by_person_time(n_experiment_persons=n_experiment_persons, mean_mu=mean_mu, std_dev_mu=std_dev_mu, treatment_effect=treatment_effect, person_time_error_sd=person_time_error_sd)
    add_imperfect_prediction(data_by_person_time, eta_mu=0.7, eta_treatment=0.6, eta_epsilon=0.8, pred_error_nu_sd=400)
    # Produce the same result for two time periods, up to decimal precision.
    print(test_diff_prediction_diff_actual_regression(data_by_person_time))
    print(test_within_person_prediction(data_by_person_time))

## Calibration

In [ ]:
# Calibrate to R^2=0.92. So 600 comes out the best fit.
std_dev_mu = 1400
for person_time_error_sd in [0, 400, 600, 800, 1600]:
    data_by_person_time = generate_data_by_person_time(n_experiment_persons=1000, mean_mu=3200, std_dev_mu=std_dev_mu,
                                                       treatment_effect=0, person_time_error_sd=person_time_error_sd, n_time_periods=2)
    print(person_time_error_sd, person_time_error_sd / std_dev_mu)
    print(test_person_fixed_effects_explained_variance(data_by_person_time))


## Run simulations

In [ ]:
n_experiment_persons=13000; mean_mu=3200; std_dev_mu=1400; treatment_effect=200; person_time_error_sd=600; n_time_periods=2
data_by_person_time = generate_data_by_person_time(n_experiment_persons=n_experiment_persons, mean_mu=mean_mu, std_dev_mu=std_dev_mu, treatment_effect=treatment_effect, person_time_error_sd=person_time_error_sd, n_time_periods=n_time_periods)

# Loop over many values of eta's, stds of components. Check how good the prediction is, and how good it is at finding treatment effect, and how well it does in the diffs prediction test, and the share of explained variance from person FE.
# And SD of actual, pred
result_df_list = []
for eta_mu in tqdm.tqdm(np.linspace(0, 1, 5), position=0, leave=True):
    for eta_treatment in np.linspace(0, 1, 5):
        for eta_epsilon in np.linspace(0, 1, 5):
            for pred_error_nu_sd in np.linspace(0, 1000, 5):
                experiment_stats_df = pd.DataFrame({'n_experiment_persons': n_experiment_persons, 'mean_mu': mean_mu,
                    'std_dev_mu': std_dev_mu, 'treatment_effect': treatment_effect, 'person_time_error_sd': person_time_error_sd,
                    'eta_mu': eta_mu, 'eta_treatment': eta_treatment, 'eta_epsilon': eta_epsilon, 'pred_error_nu_sd': pred_error_nu_sd,
                    'n_time_periods': n_time_periods}, index=[0])
                add_imperfect_prediction(data_by_person_time, eta_mu, eta_treatment, eta_epsilon, pred_error_nu_sd)
                test_stats_df = calc_prediction_stats(data_by_person_time)
                eta_mu_hat = estimate_eta_mu(data_by_person_time)
                test_stats_df['eta_mu_hat'] = eta_mu_hat
                result_df_list.append(pd.concat([experiment_stats_df, test_stats_df], axis=1))

result_df = pd.concat(result_df_list)
result_df.fillna({'treatment_t': 0}, inplace=True)
# Scale by actual treatment effect as found in the data, not the expected treatment effect.
actual_treatment_effect = reg_summary(smf.ols('actual_outcome ~ is_treatment', data=data_by_person_time).fit()).query('variable == "is_treatment[T.True]"')['coef'].iloc[0]
result_df['treatment_coef_relative'] = result_df['treatment_coef'] / actual_treatment_effect
result_df['treatment_se_relative'] = result_df['treatment_se'] / actual_treatment_effect

## Analyze simulation results

In [ ]:
labels = {'prediction_r2': 'ML Prediction R^2', 'treatment_coef_relative': 'Scaled Treatment Effect', 'treatment_t': 't-stat of Treatment Effect Coefficient', 'compression': 'Prediction Distribution Compression', 'diff_pred_coef': 'Diff-vs-diff Regression slope', 'probability': 'CDF'}
result_df['eta_T'] = result_df['eta_treatment'].astype(str)
# result_df.plot.scatter(x='eta_mu', y='treatment_t', log_y=True).show()
# result_df.plot.scatter(x='eta_mu', y='treatment_coef_relative', trendline='ols').show()
# result_df.plot.scatter(x='eta_mu', y='compression')

# Using diff_pred_coef as the predictive measure
result_df.plot.scatter(x='diff_pred_coef', y='treatment_coef_relative', color='eta_treatment', trendline='ols', width=600,
                       title='without assuming eta_treatment = eta_epsilon,<br>correlation between diffs not informative')  # Almost no correspondence.
result_df.query('eta_treatment == eta_epsilon').plot.scatter(x='diff_pred_coef', y='treatment_coef_relative', trendline='ols',
                                                             color='eta_treatment', title='Correlation between differences is indicative of treatment compression',
                                                             hover_data=['eta_treatment', 'eta_epsilon', 'pred_error_nu_sd', 'eta_mu'], width=600)  # Under this assumption, OK correspondence.
result_df.query('eta_treatment == eta_epsilon').plot.scatter(x='diff_pred_coef', error_x='diff_pred_std_err', y='eta_epsilon', trendline='ols',
                                                             color='eta_treatment', title='Actual and estimated eta_epsilon',
                                                             hover_data=['eta_treatment', 'eta_epsilon', 'pred_error_nu_sd', 'eta_mu'], width=600)  # Good correspondence.

# Distribution of coefficient with respect to theory (eta_epsilon) is the expected ~Normally distributed noise, with standard error as the standard deviation. But slightly overestimates.
result_df['diff_pred_coef_z_score'] = result_df.eval('(diff_pred_coef - eta_epsilon) / diff_pred_std_err')
px.ecdf(result_df, x='diff_pred_coef_z_score', color='eta_epsilon', labels=labels, width=600).add_hline(y=0.5, line_color='grey', line_width=2, line_dash='dash', opacity=0.5)
px.ecdf(result_df, x='diff_pred_coef_z_score', color='pred_error_nu_sd', labels=labels, width=600).add_hline(y=0.5, line_color='grey', line_width=2, line_dash='dash', opacity=0.5)
px.ecdf(result_df, x='diff_pred_coef_z_score', labels=labels, width=600).add_hline(y=0.5, line_color='grey', line_width=2, line_dash='dash', opacity=0.5)
# Slight overestimation.
result_df.groupby('pred_error_nu_sd')['diff_pred_coef_z_score'].agg(['mean', 'sem', 'std', 'median'])
result_df.groupby('eta_epsilon')['diff_pred_coef_z_score'].agg(['mean', 'sem', 'std', 'median'])

# Using diff_pred_r2 as the predictive measure
result_df.plot.scatter(x='diff_pred_r2', y='treatment_coef_relative', color='eta_treatment', trendline='ols', width=600,
                       title='without assuming eta_treatment = eta_epsilon,<br>correlation between diffs not informative')  # Almost no correspondence.
result_df.query('eta_treatment == eta_epsilon').plot.scatter(x='diff_pred_r2', y='treatment_coef_relative', trendline='ols',
                                                             color='eta_treatment', title='Correlation between differences is indicative of treatment compression',
                                                             hover_data=['eta_treatment', 'eta_epsilon', 'pred_error_nu_sd', 'eta_mu'], width=600)  # Under this assumption, OK correspondence.
result_df.query('eta_treatment == eta_epsilon').plot.scatter(x='diff_pred_r2', y='treatment_t', trendline='ols',
                                                             title='Correlation between differences is indicative of statistical power',
                                                             color='eta_treatment',
                                                             hover_data=['eta_treatment', 'pred_error_nu_sd', 'eta_mu'], width=600)
result_df.query('eta_treatment == eta_epsilon and eta_mu==0.5').plot.scatter(x='diff_pred_coef', y='treatment_coef_relative', trendline='ols',
                                                                             hover_data=['eta_treatment', 'pred_error_nu_sd', 'eta_mu'])
result_df.query('eta_mu == 0.5 and eta_treatment == eta_epsilon').plot.scatter(x='eta_treatment', y='treatment_coef_relative')
# Treatment Effect regression slightly underestimates the expected treatment effect, even factoring in eta_treatment.
add_y_equal_x_line(result_df.plot.scatter(x='eta_treatment', y='treatment_coef_relative', color='pred_error_nu_sd', labels=labels, width=600), 0, 1)
result_df['treatment_coef_z_score'] = result_df.eval('(treatment_coef_relative - eta_treatment) / treatment_se_relative')
result_df.groupby('eta_treatment')['treatment_coef_relative'].agg(['mean', 'std', 'median'])
# Discard numerical instability S.E.~0 results.
result_df.query('treatment_se_relative > 1E-10').groupby('eta_treatment')['treatment_coef_z_score'].agg(['mean', 'std', 'median'])
result_df.sort_values('treatment_se_relative').iloc[:10]


########################
# Figures in the paper #
########################
# .query('eta_treatment == eta_epsilon')
result_df.plot.scatter(x='prediction_r2', y='treatment_coef_relative', color='eta_T', trendline='ols', title='Better prediction does not guarantee more accurate treatment estimation', hover_data=['eta_T', 'pred_error_nu_sd', 'eta_mu'], width=600, labels=labels).show()
result_df.query('eta_treatment == 0').sample(frac=1).plot.scatter(color='eta_mu', y='prediction_r2', x='eta_epsilon', trendline='ols', title='Better prediction is mostly affected by person attributes', hover_data=['eta_T', 'pred_error_nu_sd', 'eta_mu'], width=600, labels=labels).show()
result_df.query('treatment_t < 1000').plot.scatter(x='prediction_r2', y='treatment_t', color='eta_treatment', trendline='ols', title='Better prediction does not guarantee more power', hover_data=['eta_T', 'pred_error_nu_sd', 'eta_mu'], width=600, labels=labels).show()

result_df.plot.scatter(x='compression', y='treatment_coef_relative', color='eta_T', trendline='ols', width=600,
                       title='Compression not very informative', labels=labels).show()  # These are only related if treatment is a big part of the variation and eta is large.

# When eta_T = eta_epsilon, our estimation of eta_epsilon does a good job of predicting the scaled treatment effect (which tends to eta_T). Slightly over-predicts.
fig = result_df.query('eta_treatment == eta_epsilon').plot.scatter(x='diff_pred_coef', y='treatment_coef_relative', # , error_x='diff_pred_std_err', trendline='ols'
                                                             color='eta_T', title='Diff-vs-diff Regression slope predicts Scaled Treatment Effect<br>when eta_T = eta_epsilon',
                                                             hover_data=['eta_T', 'eta_epsilon', 'pred_error_nu_sd', 'eta_mu'], labels=labels, width=600)
add_y_equal_x_line(fig, 0, 1).show()

# The same, without assuming eta_T = eta_epsilon. No predictive power.
result_df.plot.scatter(x='diff_pred_coef', y='treatment_coef_relative', trendline='ols',
                                                             color='eta_T', title='when eta_T != eta_epsilon, no predictive power',
                                                             hover_data=['eta_T', 'eta_epsilon', 'pred_error_nu_sd', 'eta_mu'], labels=labels, width=600).show()


# The prediction of differences cancels out the person FE, so what's left is eta_epsilon and exogenous prediction error:
result_df.query('eta_treatment == eta_epsilon').plot.scatter(x='pred_error_nu_sd', y='diff_pred_r2', trendline='ols', color='eta_epsilon',
                                                             title='Person FE cancel out, so eta_epsilon and nu is what is left', width=600,
                                                             hover_data=['eta_T', 'pred_error_nu_sd', 'eta_mu'], labels=labels).show()  # Under this assumption, OK correspondence.


# Estimating eta_mu
fig = result_df.plot.scatter(x='eta_mu_hat', y='eta_mu') # , error_x='diff_pred_std_err', trendline='ols'
                            # color='eta_T', title='Diff-vs-diff Regression slope predicts Scaled Treatment Effect<br>when eta_T = eta_epsilon',
                            # hover_data=['eta_T', 'eta_epsilon', 'pred_error_nu_sd', 'eta_mu'], labels=labels, width=600)
add_y_equal_x_line(fig, 0, 1).show()
